In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import time
import cv2
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

IMG_SIZE = 128


#код для нейромережі взято з курсу Convolutional Neural Networks на платформі Coursera

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


In [9]:
# GRADED FUNCTION: identity_block

def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [10]:
# GRADED FUNCTION: convolutional_block

def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(F2, (f, f), strides = (1,1), name = conv_name_base + '2b', padding = 'same',  kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(F3, (1, 1), strides = (1,1), name = conv_name_base + '2c', padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X

In [11]:
# GRADED FUNCTION: ResNet50

def ResNet(input_shape = (128, 128, 3), classes = 4):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###
    '''
    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
    '''

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D([2, 2], name = "avg_pool")(X)
    
    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [12]:
IMG_SIZE = 128

model = ResNet(input_shape = (IMG_SIZE, IMG_SIZE, 3), classes = 4)

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [87]:
#зчитуємо дані
X = np.zeros((363, IMG_SIZE, IMG_SIZE, 3))
Y = np.zeros((363, 4))
for i in range(121):
    img = cv2.imread(f"img/{i}.jpg")
    x = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
    x = np.expand_dims(x, axis=0)
    x = x/255.0
    X[i] = x
    if(i <= 102):
        Y[i][0] = 1
    else:
        Y[i][3] = 1
for i in range(121):
    img = cv2.imread(f"img2/{i}.jpg")
    x = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
    x = np.expand_dims(x, axis=0)
    x = x/255.0
    X[i+121] = x
    if(i <= 100):
        Y[i+121][1] = 1
    else:
        Y[i+121][3] = 1
for i in range(121):
    img = cv2.imread(f"img3/{i}.jpg")
    x = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
    x = np.expand_dims(x, axis=0)
    x = x/255.0
    X[i+242] = x
    if(i <= 94):
        Y[i+242][2] = 1
    else:
        Y[i+242][3] = 1        

        
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0, test_size=0.2)



# Convert training and test labels to one hot matrices
#Y_train = convert_to_one_hot(Y_train_orig, 6).T
#Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))


number of training examples = 290
number of test examples = 73
X_train shape: (290, 128, 128, 3)
Y_train shape: (290, 4)
X_test shape: (73, 128, 128, 3)
Y_test shape: (73, 4)


In [88]:
#навчаємо нейромережу
model.fit(X_train, Y_train, epochs = 5, batch_size = 32)

Epoch 1/5
10/10 [==============================] - 17s 2s/step - loss: 1.0690 - accuracy: 0.8897
Epoch 2/5
10/10 [==============================] - 17s 2s/step - loss: 0.6653 - accuracy: 0.9241
Epoch 3/5
10/10 [==============================] - 18s 2s/step - loss: 0.3645 - accuracy: 0.9310
Epoch 4/5
10/10 [==============================] - 17s 2s/step - loss: 0.1308 - accuracy: 0.9724
Epoch 5/5
10/10 [==============================] - 17s 2s/step - loss: 0.0192 - accuracy: 0.9931


In [89]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

3/3 [==============================] - 0s 129ms/step - loss: 25.5910 - accuracy: 0.3425
Loss = 25.590974807739258
Test Accuracy = 0.34246575832366943


In [27]:
#прохід по конкретному зображенню
img = cv2.imread(f"img/0.jpg")
x = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
x = np.expand_dims(x, axis=0)
x = x/255.0
print('Input image shape:', x.shape)
#my_image = scipy.misc.imread(img_path)
#imshow(my_image)
print("class prediction vector [p(0), p(1), p(2), p(3)] = ")
print(model.predict(x))

Input image shape: (1, 128, 128, 3)
class prediction vector [p(0), p(1), p(2), p(3)] = 
[[6.0505408e-01 6.1265113e-05 1.7438782e-02 3.7744597e-01]]


In [92]:
#прохід по тестовим даним та збір метрик
import time
t1 = 0
t2 = 0
t3 = 0
t4 = 0
a1 = 0
a2 = 0
a3 = 0
a4 = 0
times = 0
for i in range(X_test.shape[0]):
    start_time = time.time()
    p = model.predict(np.expand_dims(X_test[i], axis=0))
    times += (time.time() - start_time)
    #for j in range(4):
        #if p[j] == np.max(p):
    yp = np.argmax(p)
    yt = np.argmax(Y_test[i])
    #print(f"{i+1}: {yp} {yt}")
    if yp == yt == 0: t1 += 1
    elif yp == yt == 1: t2 += 1
    elif yp == yt == 2: t3 += 1
    elif yp == yt == 3: t4 += 1
        
    if yp == 0: a1 += 1
    elif yp == 1: a2 += 1
    elif yp == 2: a3 += 1
    elif yp == 3: a4 += 1

sY = np.sum(Y_test, axis = 0)
print(f"detected car:\t\t{t1/sY[0]}")
print(f"detected stand:\t\t{t2/sY[1]}")
print(f"detected dosimeter:\t{t3/sY[2]}")
print(f"nothing detected:\t{t4/sY[3]}\n")

print(f"false positive car:\t{(a1 - t1)/a1}")
print(f"false positive stand:\t{(a2 - t2)/a2}")
print(f"false positive dosimeter:{(a3 - t3)/a3}")
print(f"false negative:\t\t{(a4 - t4)/a4}\n")

print(f"mean time:\t\t{times/X.shape[0]}s")

detected car:		0.8333333333333334
detected stand:		0.8
detected dosimeter:	0.5882352941176471
nothing detected:	1.0

false positive car:	0.047619047619047616
false positive stand:	0.0
false positive dosimeter:0.09090909090909091
false negative:		0.52

mean time:		0.009447786105237388s


In [80]:
#зберігаємо нейромережу
model.save('ReNet')

INFO:tensorflow:Assets written to: ReNet\assets


In [91]:
#завантажуємо нейпромережу
model = load_model('ReNet')

In [76]:

X1 = np.zeros((30, IMG_SIZE, IMG_SIZE, 3))
Y1 = np.zeros((30, 4))
for i in range(30):
    img = cv2.imread(f"img/{121+i}.jpg")
    x = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
    x = np.expand_dims(x, axis=0)
    x = x/255.0
    X1[i] = x
    if i < 21:
        Y1[i][0] = 1
    else:
        Y1[i][3] = 1


In [77]:
X2 = np.concatenate((X, X1), axis = 0)
Y2 = np.concatenate((Y, Y1), axis = 0)

In [78]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X2, Y2, random_state=0, test_size=0.2)